<a href="https://colab.research.google.com/github/solobala/DLL30/blob/main/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_2_%D0%9C%D0%BD%D0%BE%D0%B3%D0%BE%D1%81%D0%BB%D0%BE%D0%B9%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch

In [3]:
import numpy as np

## MLP

## Данные

In [4]:
import torchvision as tv

In [5]:
import pandas as pd
import numpy as np

In [6]:
import time

In [7]:
BATCH_SIZE=256

In [8]:
train_dataset = tv.datasets.MNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

100%|██████████| 9912422/9912422 [00:00<00:00, 73529239.19it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 95382436.08it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 17807297.60it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6034377.18it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [9]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

## Модель

In [10]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

In [11]:
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)

In [12]:
loss = torch.nn.CrossEntropyLoss()
trainer = torch.optim.SGD(model.parameters(), lr=.01)
num_epochs = 10

In [13]:
import time

In [14]:
def train_model():  # здеаь без батчей, похоже
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        # Обучение
        model.train()
        for X, y in train:
            trainer.zero_grad() # обнуляем гнрадиенты
            y_pred = model(X) # делаем прогноз
            l = loss(y_pred, y) # Считаем ф-ю потер, y_pred Содержит 10 значений
            l.backward() # считаем градиенты
            trainer.step()
            train_loss += l.item() # Накапливаем потерм
            train_acc += (y_pred.argmax(dim=1) == y).sum().item() # считаем accuracy
            train_iters += 1 # сколько значений прошли
            train_passed += len(X) # Сколько осталось
        # Оценка на тесте
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
        # Печатаем среднюю accuracy  И loss за эпоху для train и test
        print("ep: {}, taked: {:.3f}, train_loss: {}, train_acc: {}, test_loss: {}, test_acc: {}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [15]:
train_model()

ep: 0, taked: 8.802, train_loss: 2.047048894902493, train_acc: 0.5540666666666667, test_loss: 1.698171153664589, test_acc: 0.7148
ep: 1, taked: 9.984, train_loss: 1.3437845554757626, train_acc: 0.7627666666666667, test_loss: 1.0160796761512756, test_acc: 0.8103
ep: 2, taked: 9.974, train_loss: 0.8693519701348974, train_acc: 0.8238333333333333, test_loss: 0.716173668205738, test_acc: 0.8467
ep: 3, taked: 9.361, train_loss: 0.6655285027432949, train_acc: 0.8495166666666667, test_loss: 0.5795899279415607, test_acc: 0.8653
ep: 4, taked: 9.518, train_loss: 0.5628785508110168, train_acc: 0.8645666666666667, test_loss: 0.5032656487077475, test_acc: 0.8764
ep: 5, taked: 9.989, train_loss: 0.5015312483970155, train_acc: 0.8743833333333333, test_loss: 0.45475257001817226, test_acc: 0.8837
ep: 6, taked: 9.547, train_loss: 0.4607993708646044, train_acc: 0.88115, test_loss: 0.42134969830513, test_acc: 0.8886
ep: 7, taked: 9.218, train_loss: 0.43178728990098264, train_acc: 0.88615, test_loss: 0.3969

## Практика. Попробуйте заменить SGD на Adam и RMSProp. Увеличится ли скорость сходимости?

In [16]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)

In [17]:
trainer = torch.optim.Adam(model.parameters(), lr=.01)  # Переобучение начиная с 3 эпохи, но accuracy существенно выше
train_model()

ep: 0, taked: 10.225, train_loss: 0.24826281508232684, train_acc: 0.9248, test_loss: 0.20014091844204812, test_acc: 0.935
ep: 1, taked: 10.318, train_loss: 0.10771608574395167, train_acc: 0.9668, test_loss: 0.11128803972969763, test_acc: 0.9662
ep: 2, taked: 9.985, train_loss: 0.07204192415414815, train_acc: 0.97705, test_loss: 0.11917600364977261, test_acc: 0.963
ep: 3, taked: 9.495, train_loss: 0.05897880765668889, train_acc: 0.9806666666666667, test_loss: 0.1359688817843562, test_acc: 0.9655
ep: 4, taked: 10.503, train_loss: 0.051842852607567574, train_acc: 0.9834, test_loss: 0.12848851835551614, test_acc: 0.9657
ep: 5, taked: 10.430, train_loss: 0.04894958788459684, train_acc: 0.9841833333333333, test_loss: 0.14251997497631236, test_acc: 0.9686
ep: 6, taked: 9.846, train_loss: 0.04483805794808816, train_acc: 0.9863, test_loss: 0.1364627841973743, test_acc: 0.97
ep: 7, taked: 9.998, train_loss: 0.042376151451088015, train_acc: 0.9869166666666667, test_loss: 0.1314064044439874, test_

## Практика. Попробуйте сделать больше слоёв в сети. Увеличится ли качество?

In [18]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

In [20]:
trainer = torch.optim.Adam(model.parameters(), lr=.01) # Переобучение с 3 эпохи, accuracy выросла - модель слишком сложная
train_model()

ep: 0, taked: 13.687, train_loss: 0.067689781765295, train_acc: 0.98395, test_loss: 0.1663854336620716, test_acc: 0.9668
ep: 1, taked: 13.533, train_loss: 0.06178531164967237, train_acc: 0.9844333333333334, test_loss: 0.16196658518271861, test_acc: 0.9715
ep: 2, taked: 13.303, train_loss: 0.05868520388389601, train_acc: 0.9849166666666667, test_loss: 0.15623091034649406, test_acc: 0.9688
ep: 3, taked: 13.602, train_loss: 0.04957530174523275, train_acc: 0.9871833333333333, test_loss: 0.14269433827430475, test_acc: 0.9747
ep: 4, taked: 13.461, train_loss: 0.045946947804236034, train_acc: 0.9882666666666666, test_loss: 0.15207495358299639, test_acc: 0.9756
ep: 5, taked: 13.540, train_loss: 0.04109842350007966, train_acc: 0.9893333333333333, test_loss: 0.18504840555250368, test_acc: 0.9753
ep: 6, taked: 13.680, train_loss: 0.04155927098971083, train_acc: 0.98955, test_loss: 0.18152296735642537, test_acc: 0.9724
ep: 7, taked: 13.572, train_loss: 0.04103428056300122, train_acc: 0.98976666666

## Практика. Попробуйте добавить регуляризацию, dropout и/или batchnorm-слои. Увеличится ли качество?

In [21]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(512),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(256),
    torch.nn.Linear(256, 128),
    torch.nn.ReLU(),
    torch.nn.BatchNorm1d(128),
    torch.nn.Linear(128, 10)
)

In [22]:
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 15.285, train_loss: 0.19569876264146668, train_acc: 0.9395, test_loss: 0.15486095857340842, test_acc: 0.953
ep: 1, taked: 12.630, train_loss: 0.09874235736800635, train_acc: 0.9695, test_loss: 0.11410284111334476, test_acc: 0.9652
ep: 2, taked: 12.950, train_loss: 0.07235159838294729, train_acc: 0.9767666666666667, test_loss: 0.1321139476960525, test_acc: 0.9618
ep: 3, taked: 13.119, train_loss: 0.05449661841130558, train_acc: 0.98265, test_loss: 0.10157693566652597, test_acc: 0.9702
ep: 4, taked: 13.162, train_loss: 0.04094244216846183, train_acc: 0.9869166666666667, test_loss: 0.11625943237304455, test_acc: 0.9681
ep: 5, taked: 13.262, train_loss: 0.03654788486074973, train_acc: 0.9884333333333334, test_loss: 0.12723873125360113, test_acc: 0.9647
ep: 6, taked: 13.276, train_loss: 0.03306745445117989, train_acc: 0.9891, test_loss: 0.11397605922684306, test_acc: 0.9706
ep: 7, taked: 12.747, train_loss: 0.02679457537572276, train_acc: 0.9911833333333333, test_loss: 0.10119

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784, 2560),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(2560, 1280),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1280, 640),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(640, 10)
)

In [ ]:
trainer = torch.optim.Adam(model.parameters(), lr=.01)
train_model()

ep: 0, taked: 53.651, train_loss: 0.9062292760991035, train_acc: 0.745, test_loss: 0.5461626082658768, test_acc: 0.8892
ep: 1, taked: 53.853, train_loss: 0.690170981782548, train_acc: 0.81215, test_loss: 0.3796499317511916, test_acc: 0.9073
ep: 2, taked: 58.531, train_loss: 0.708586008117554, train_acc: 0.8084, test_loss: 0.3357968518510461, test_acc: 0.9188
ep: 3, taked: 81.395, train_loss: 0.7057550713102868, train_acc: 0.8098666666666666, test_loss: 0.3372530031017959, test_acc: 0.9165
ep: 4, taked: 74.981, train_loss: 0.7496358336286342, train_acc: 0.7984, test_loss: 0.3336694069206715, test_acc: 0.9192
ep: 5, taked: 75.582, train_loss: 0.7321985492046843, train_acc: 0.8017, test_loss: 0.326321928948164, test_acc: 0.9173
ep: 6, taked: 75.639, train_loss: 0.7420422671957219, train_acc: 0.7959, test_loss: 0.3461838087067008, test_acc: 0.921
ep: 7, taked: 76.033, train_loss: 0.7599228668720164, train_acc: 0.7928166666666666, test_loss: 0.3387015956453979, test_acc: 0.9167
ep: 8, taked